In [84]:
import os
import exifread
import pandas as pd
# Allows all columns to be shown in the dataframe at the bottom of this notebook
pd.set_option('display.max_columns', None)

### Locate the JPEGs whose metadata is to be analyzed. Since the operation is performed on one image, a variable is set to the path. If there were multiple images, the path for each images is appended to a list.

In [80]:
start_search = './'

for r, d, f in os.walk(start_search):
    for file in f:
        if '.JPG' in file:
            image_path = os.path.join(r, file)

### Searching for and parsing the XMP (Adobe's Extensible Metadata Platform) data in the jpeg. 

In [81]:
photo = open(image_path, 'rb')
data = photo.read()

xmp_start = data.find(b'<x:xmpmeta')
xmp_end = data.find(b'</x:xmpmeta')
xmp_string = data[xmp_start:xmp_end+12]

# Seperate each xmp field
xmp_data_list = str(xmp_string).split()

# Parse each seperated field and populate dictionary
xmp_dict = dict()
for field in xmp_data_list:
    if '=' in field:
        key, value = field.split('=')
        xmp_dict[key] = value
        

xmp_df = pd.DataFrame(data=xmp_dict, index=[0])

photo.close()

### Utilizing exifread python library, the exif metadata of the jpeg is accessed.

In [82]:
f = open(image_path, 'rb')

tags = exifread.process_file(f)

exif_dict = dict()

for tag in tags.keys():
    if tag not in ('JPEGThumbnail'):
        exif_dict[tag] = tags[tag]
        
        
exif_df = pd.DataFrame(data=exif_dict, index=[0])

f.close()

### The xmp pandas dataframe is combined with the exif metadata dataframe to create a single dataframe (which can be made to a csv) of jpeg metadata.

In [83]:
final_df = pd.concat([xmp_df, exif_df], axis=1, sort=False)

final_df.head()

,xmlns:x,xmlns:rdf,rdf:about,xmlns:tiff,xmlns:exif,xmlns:xmp,xmlns:xmpMM,xmlns:dc,xmlns:crs,xmlns:drone-dji,xmp:ModifyDate,xmp:CreateDate,tiff:Make,tiff:Model,dc:format,drone-dji:AbsoluteAltitude,drone-dji:RelativeAltitude,drone-dji:GimbalRollDegree,drone-dji:GimbalYawDegree,drone-dji:GimbalPitchDegree,drone-dji:FlightRollDegree,drone-dji:FlightYawDegree,drone-dji:FlightPitchDegree,drone-dji:CamReverse,drone-dji:GimbalReverse,drone-dji:SelfData,crs:Version,crs:HasSettings,crs:HasCrop,crs:AlreadyApplied,Image ImageDescription,Image Make,Image Model,Image Orientation,Image XResolution,Image YResolution,Image ResolutionUnit,Image Software,Image DateTime,Image YCbCrPositioning,Image ExifOffset,GPS GPSVersionID,GPS GPSLatitudeRef,GPS GPSLatitude,GPS GPSLongitudeRef,GPS GPSLongitude,GPS GPSAltitudeRef,GPS GPSAltitude,Image GPSInfo,Image XPComment,Image XPKeywords,Thumbnail Compression,Thumbnail XResolution,Thumbnail YResolution,Thumbnail ResolutionUnit,Thumbnail JPEGInterchangeFormat,Thumbnail JPEGInterchangeFormatLength,EXIF ExposureTime,EXIF FNumber,EXIF ExposureProgram,EXIF ISOSpeedRatings,EXIF ExifVersion,EXIF DateTimeOriginal,EXIF DateTimeDigitized,EXIF ComponentsConfiguration,EXIF CompressedBitsPerPixel,EXIF ShutterSpeedValue,EXIF ApertureValue,EXIF ExposureBiasValue,EXIF MaxApertureValue,EXIF SubjectDistance,EXIF MeteringMode,EXIF LightSource,EXIF Flash,EXIF FocalLength,EXIF MakerNote,EXIF FlashPixVersion,EXIF ColorSpace,EXIF ExifImageWidth,EXIF ExifImageLength,Interoperability InteroperabilityIndex,Interoperability InteroperabilityVersion,EXIF InteroperabilityOffset,EXIF ExposureIndex,EXIF FileSource,EXIF SceneType,EXIF CustomRendered,EXIF ExposureMode,EXIF WhiteBalance,EXIF DigitalZoomRatio,EXIF FocalLengthIn35mmFilm,EXIF SceneCaptureType,EXIF GainControl,EXIF Contrast,EXIF Saturation,EXIF Sharpness,EXIF SubjectDistanceRange,EXIF BodySerialNumber
0,"""adobe:ns:meta/"">\n","""http://www.w3.org/1999/02/22-rdf-syntax-ns#"">\n","""DJI","""http://ns.adobe.com/tiff/1.0/""\n","""http://ns.adobe.com/exif/1.0/""\n","""http://ns.adobe.com/xap/1.0/""\n","""http://ns.adobe.com/xap/1.0/mm/""\n","""http://purl.org/dc/elements/1.1/""\n","""http://ns.adobe.com/camera-raw-settings/1.0/""\n","""http://www.dji.com/drone-dji/1.0/""\n","""2018-01-21""\n","""2018-01-21""\n","""DJI""\n","""FC6310""\n","""image/jpg""\n","""+279.26""\n","""+64.30""\n","""+0.00""\n","""-84.40""\n","""-35.60""\n","""+6.20""\n","""-86.40""\n","""-1.30""\n","""0""\n","""0""\n","""59a9a28ee041f2063bbfadb42.55.1""\n","""7.0""\n","""False""\n","""False""\n","""False"">\n",DCIM\100MEDIA\DJI_0024.JPG,DJI,FC6310,Horizontal (normal),72,72,Pixels/Inch,v01.05.1577,2018:01:21 09:30:10,Centered,182,"[2, 3, 0, 0]",N,"[34, 32, 61861/10000]",W,"[115, 28, 144117/10000]",0,34907/125,686,"[84, 0, 121, 0, 112, 0, 101, 0, 61, 0, 78, 0, 44, 0, 32, 0, 77, 0, 111, 0, ... ]","[118, 0, 48, 0, 49, 0, 46, 0, 48, 0, 53, 0, 46, 0, 49, 0, 53, 0, 55, 0, ... ]",JPEG (old-style),72,72,Pixels/Inch,3060,10157,1/500,28/5,Program Normal,100,0230,2018:01:21 09:30:10,2018:01:21 09:30:10,CrCbY,48281/14592,-8965/-1000,497/100,0,297/100,0,CenterWeightedAverage,Unknown,No flash function,44/5,"[33, 0, 1, 0, 2, 0, 4, 0, 0, 0, 68, 74, 73, 0, 2, 0, 1, 0, 4, 0, ... ]",0010,sRGB,4864,3648,R98,"[48, 49, 48, 48]",656,0/0,Digital Camera,Directly Photographed,Normal,Auto Exposure,Manual,0/0,24,Standard,None,Normal,Normal,Hard,0,258099607acafe0014a74787b55c7098
